Now this file will all the data cleaning done on the dataset according to the findings of the EDA.


In [263]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_raw = pd.read_csv("electricity_consumption_optimization_dataset.csv")

df = df_raw.copy()
print('Shape:- ', df.shape)
df.info()   


Shape:-  (52585, 40)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52585 entries, 0 to 52584
Data columns (total 40 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Timestamp                            52585 non-null  object 
 1   Building Type                        52585 non-null  object 
 2   Energy Consumption (kWh)             52585 non-null  float64
 3   Temperature (°C)                     52585 non-null  float64
 4   Humidity (%)                         52585 non-null  float64
 5   Occupancy Rate (%)                   52585 non-null  float64
 6   Lighting Consumption (kWh)           52585 non-null  float64
 7   HVAC Consumption (kWh)               52585 non-null  float64
 8   Energy Price ($/kWh)                 52585 non-null  float64
 9   Carbon Emission Rate (g CO2/kWh)     52585 non-null  float64
 10  Power Factor                         52585 non-null  float64
 11  Voltage

In [264]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['hour'] = df['Timestamp'].dt.hour
df['day'] = df['Timestamp'].dt.day
df['month'] = df['Timestamp'].dt.month
df.drop('Timestamp', axis=1, inplace=True)
df


,Building Type,Energy Consumption (kWh),Temperature (°C),Humidity (%),Occupancy Rate (%),Lighting Consumption (kWh),HVAC Consumption (kWh),Energy Price ($/kWh),Carbon Emission Rate (g CO2/kWh),Power Factor,...,Electric Vehicle Charging Status,Building Orientation,IoT Sensor Count,Thermal Comfort Index,Energy Savings Potential (%),Peak Demand Reduction Indicator,Carbon Emission Reduction Category,hour,day,month
0,Residential,74.679912,31.357437,62.472886,49.293544,9.892054,9.073339,0.053297,341.764320,0.932078,...,0,South,21.425365,80.807871,14.115263,0,Moderate Reduction,0,1,1
1,Industrial,46.592761,30.229121,63.067770,65.039117,11.063696,26.487881,0.019031,427.270057,0.984152,...,1,South,34.385339,79.679998,4.107683,0,Moderate Reduction,1,1,1
2,Commercial,58.836657,19.182581,65.030972,-16.599946,0.582297,10.385565,0.060282,278.064969,0.800676,...,0,North,67.588352,84.571954,4.130955,0,Moderate Reduction,2,1,1
3,Residential,53.586516,16.700048,67.405643,27.398152,3.580005,8.199913,0.209318,691.906306,0.921343,...,0,South,37.714465,95.953736,8.681566,0,No Reduction,3,1,1
4,Residential,37.800448,29.620051,55.069770,74.220380,17.823608,12.268133,0.225335,620.526286,0.901588,...,0,North,36.321629,68.227000,19.631869,0,Moderate Reduction,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52580,Residential,78.697373,18.669766,76.145160,86.565125,11.475367,13.489796,0.119900,499.016888,0.869080,...,0,South,59.425428,70.658451,14.399685,0,Low Reduction,20,31,12
52581,Residential,59.400463,24.814296,93.473230,51.380712,18.418070,25.319140,0.180057,344.329532,1.058497,...,0,East,50.490422,89.069748,20.619243,0,Low Reduction,21,31,12
52582,Residential,65.395535,3.952222,75.281979,49.625698,17.637973,19.546613,0.168849,317.509517,0.768559,...,0,South,74.773075,70.760759,15.645947,0,Moderate Reduction,22,31,12
52583,Residential,58.505085,12.342418,54.181047,87.792609,4.205146,-4.032314,0.139890,183.725258,0.858071,...,0,South,75.925739,73.370549,11.999294,0,No Reduction,23,31,12


Removing negative energy values ( Since energy can't be in -ve)

first i tried to do this but then it removed almost 7k+ rows 
so now i am clipping them instead of removing

In [265]:
energy_cols = ['Energy Consumption (kWh)', 'Lighting Consumption (kWh)',
               'HVAC Consumption (kWh)', 'Local Energy Production (kWh)',
               'Room-Level Energy Consumption (kWh)', 'Zonal Heating/Cooling Data (kWh)',
               'Smart Plug Usage (kWh)', 'Historical Energy Consumption (kWh)']
for col in energy_cols:
    df[col] = df[col].clip(lower=0)
print('Shape', df.shape)


Shape (52585, 42)


Fixing occupancy rate ( setting the rabge of it between 0 to 100)

In [266]:
df["Occupancy Rate (%)"] = df["Occupancy Rate (%)"].clip(0, 100)
print(df['Occupancy Rate (%)'])

0        49.293544
1        65.039117
2         0.000000
3        27.398152
4        74.220380
           ...    
52580    86.565125
52581    51.380712
52582    49.625698
52583    87.792609
52584    67.929317
Name: Occupancy Rate (%), Length: 52585, dtype: float64


In [267]:
pct_cols = ['Window-to-Wall Ratio (%)',
            'Energy Savings Target (%)', 'Energy Savings Potential (%)']
for col in pct_cols:
    df[col] = df[col].clip(0, 100)

x = df[[
    "Window-to-Wall Ratio (%)",
    "Energy Savings Target (%)",
    "Energy Savings Potential (%)"
]]
print(x)



       Window-to-Wall Ratio (%)  Energy Savings Target (%)  \
0                     32.260201                  16.208574   
1                     37.106190                  16.081108   
2                     22.450161                  17.463861   
3                     30.163232                  14.132527   
4                     43.700672                  20.960512   
...                         ...                        ...   
52580                 24.990368                  18.656061   
52581                 27.126211                  27.028035   
52582                 34.938495                  22.125447   
52583                 22.306715                  11.750403   
52584                 16.614175                  16.483761   

       Energy Savings Potential (%)  
0                         14.115263  
1                          4.107683  
2                          4.130955  
3                          8.681566  
4                         19.631869  
...                        

Since humidity cant be negative so instead removing clipping it and also the power factor since its a magnitude
and fixing all the other negative values

In [268]:
df['Humidity (%)'] = df['Humidity (%)'].clip(0, 100)
df['Power Factor'] = df['Power Factor'].clip(0, 1)
df['Energy Price ($/kWh)'] = df['Energy Price ($/kWh)'].clip(lower=0)
df['Building Size (m²)'] = df['Building Size (m²)'].clip(lower=0)
df['Solar Irradiance (W/m²)'] = df['Solar Irradiance (W/m²)'].clip(lower=0)
df['Water Usage (liters)'] = df['Water Usage (liters)'].clip(lower=0)
df['Building Age (years)'] = df['Building Age (years)'].clip(lower=0)
df['Equipment Age (years)'] = df['Equipment Age (years)'].clip(lower=0)
df['Energy Efficiency Rating'] = df['Energy Efficiency Rating'].clip(lower=0)
df['Insulation Quality Score'] = df['Insulation Quality Score'].clip(lower=0)
df['Reactive Power (kVARh)'] = df['Reactive Power (kVARh)'].clip(lower=0)
df['IoT Sensor Count'] = df['IoT Sensor Count'].clip(lower=0)
df['Energy Efficiency Rating'] = df['Energy Efficiency Rating'].clip(upper=100)
df['Thermal Comfort Index'] = df['Thermal Comfort Index'].clip(upper=100)

Outlier analysis:- first i tried to implement the basic iqr method but since it drops the rows so almost 66% of data was removed like only 18k rows remained and also since there are more than 40 features therefore it is a problem 

then i also thought of using  z score but since the features are many, i can loose the variance in the dataset and in clustering part i will get problems.

so finally sticking to iqr but instead of dropping i will clip them and by standard scaler i will scale them and clutering will be good on 52k rows

The main problem is data is synthetic gaussian data

In [269]:
outlier_cols = ['Building Size (m²)', 'Carbon Emission Rate (g CO2/kWh)',
                'Solar Irradiance (W/m²)', 'Historical Energy Consumption (kWh)',
                'Water Usage (liters)']   # added Water Usage

# ---- ADD DIAGNOSTIC FOR OUTLIER CLIPPING ----
# Capture original values before clipping
original_outlier_vals = df[outlier_cols].copy()

for col in outlier_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower, upper)
# After clipping, print how many values changed and variance impact
for col in outlier_cols:
    modified = (original_outlier_vals[col] != df[col]).sum()
    print(f"{col}: {modified} values modified ({modified/len(df)*100:.2f}%)")
    print(f"  Variance before: {original_outlier_vals[col].var():.2f}")
    print(f"  Variance after : {df[col].var():.2f}")
# ---------------------------------------------

print("Shape after outlier removal:", df.shape)

Building Size (m²): 203 values modified (0.39%)
  Variance before: 250935.59
  Variance after : 249007.63
Carbon Emission Rate (g CO2/kWh): 344 values modified (0.65%)
  Variance before: 22779.12
  Variance after : 22516.03
Solar Irradiance (W/m²): 188 values modified (0.36%)
  Variance before: 9989.67
  Variance after : 9920.11
Historical Energy Consumption (kWh): 200 values modified (0.38%)
  Variance before: 630.29
  Variance after : 625.67
Water Usage (liters): 181 values modified (0.34%)
  Variance before: 2483.64
  Variance after : 2467.60
Shape after outlier removal: (52585, 42)


Now doing encoding on categorical cols to make them numeric and useful for clustering and here i have used one hot encoding instead of label encoding

In [270]:
cat_cols = ['Building Type', 'Occupancy Schedule', 'Building Orientation',
            'Carbon Emission Reduction Category']
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)


Cynical encoding for the time variables 
hour (0–23), day (1–31), and month (1–12) are currently treated as linear numbers. StandardScaler assumes that the difference between 23 and 0 is 23 units, but in reality they are only 1 hour apart. This introduces an artificial distance that can mislead clustering.



In [271]:
# ---- ADD CYCLICAL ENCODING ----
# Hour (0-23) -> sin/cos
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# Month (1-12) -> convert to 0-11, then sin/cos
df['month_sin'] = np.sin(2 * np.pi * (df['month'] - 1) / 12)
df['month_cos'] = np.cos(2 * np.pi * (df['month'] - 1) / 12)

# Optional: day of month (1-31) – good to include, though cycle length varies
df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)

# Drop the original linear time columns (no longer needed)
df.drop(['hour', 'day', 'month'], axis=1, inplace=True)
# ---------------------------------

Now doing feature scaling by using standard scaler 

In [272]:
# Create a list of the cyclical columns you just generated
cyclical_cols = ['hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'day_sin', 'day_cos']

# Re-identify binary columns
binary_cols = [col for col in df.columns if df[col].dropna().isin([0,1]).all()]

# CORRECTED: Subtract BOTH binary and cyclical columns from the scaling list
continuous_cols = df.columns.difference(binary_cols + cyclical_cols)

Loading the cleaned & unscaled data for feature engineering 

In [273]:
# Save the cleaned unscaled dataframe
df.to_csv("cleaned_unscaled.csv", index=False)
print("Saved cleaned_unscaled.csv")

Saved cleaned_unscaled.csv


In [274]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[continuous_cols] = scaler.fit_transform(df[continuous_cols])

print("Final cleaned shape:", df_scaled.shape)

Final cleaned shape: (52585, 50)


In [275]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Energy Consumption (kWh),52585.0,5.406488e+01,20.237984,0.000000,40.378669,5.422576e+01,67.713406,136.499217
Temperature (°C),52585.0,2.122291e+01,10.028437,-20.737576,14.508595,2.116508e+01,27.984117,63.394893
Humidity (%),52585.0,6.125059e+01,19.423117,0.000000,47.896731,6.148884e+01,74.925621,100.000000
Occupancy Rate (%),52585.0,5.851110e+01,27.701146,0.000000,38.667284,5.961371e+01,80.647693,100.000000
Lighting Consumption (kWh),52585.0,1.118470e+01,5.032844,0.000000,7.715759,1.116067e+01,14.595504,33.226991
HVAC Consumption (kWh),52585.0,1.609289e+01,6.948060,0.000000,11.356539,1.607036e+01,20.783193,43.454961
Energy Price ($/kWh),52585.0,1.561452e-01,0.050063,0.000000,0.122649,1.564739e-01,0.189819,0.365671
Carbon Emission Rate (g CO2/kWh),52585.0,4.297557e+02,150.053434,20.721165,327.388796,4.292805e+02,531.833883,838.501514
Power Factor,52585.0,8.982410e-01,0.085265,0.433982,0.840439,9.078930e-01,0.976403,1.000000
Voltage Levels (V),52585.0,2.312134e+02,10.033883,189.182698,224.441780,2.311797e+02,237.960719,275.813114


Checking Multicollinearity

In [276]:
import numpy as np

corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

high_corr = [column for column in upper.columns if any(upper[column] > 0.95)]
print("Highly correlated columns:", high_corr)

Highly correlated columns: []


In [277]:
static_cols_raw = [
    'Building Type',
    'Building Orientation',
    'Occupancy Schedule',
    'Carbon Emission Reduction Category'
]
static_cols_numeric = [
    'Building Size (m²)',
    'Building Age (years)',
    'Equipment Age (years)',
    'Insulation Quality Score',
    'IoT Sensor Count'
]

# Build a temporary table with the original categorical and the current numeric static columns
temp = df_raw[static_cols_raw].copy()
for col in static_cols_numeric:
    temp[col] = df[col].values

print("Total rows:", len(temp))
unique_combo = temp.drop_duplicates().shape[0]
print("Unique structural combinations:", unique_combo)

Total rows: 52585
Unique structural combinations: 52585


In [278]:
# Shows the first 5 rows, but flipped vertically
print(df_scaled.head().T)

                                                           0         1  \
Energy Consumption (kWh)                             1.01864 -0.369216   
Temperature (°C)                                    1.010589  0.898076   
Humidity (%)                                         0.06293  0.093558   
Occupancy Rate (%)                                 -0.332753  0.235661   
Lighting Consumption (kWh)                         -0.256845 -0.024043   
HVAC Consumption (kWh)                               -1.0103  1.496113   
Energy Price ($/kWh)                               -2.054376 -2.738829   
Carbon Emission Rate (g CO2/kWh)                   -0.586406 -0.016565   
Power Factor                                        0.396852  1.007594   
Voltage Levels (V)                                  0.626733 -0.997955   
Reactive Power (kVARh)                              1.289212  0.795916   
Power Outage Indicator                                     0         0   
Indoor Temperature (°C)               